In [ ]:
from database import Database

database = Database(
    'maya-dev.cxb0ogdnnu26.ap-southeast-1.rds.amazonaws.com',
    'maya_db',
    'root',
    'maya))&))&',
    'utf8mb4'
)
connection = database.connect_with_pymysql()

if connection:
    with connection.cursor() as cursor:
        try:
            sql = "select questions.id as qid, questions.body as qbody, questions_tags.tag_id as qtag from questions inner join questions_tags on questions.id=questions_tags.question_id where questions.source='robi'"
            cursor.execute(sql)
            robi_qs = cursor.fetchall()
        finally:
            connection.close()
print len(robi_qs)

In [ ]:
def utf8_decode(data):
    try:
        return data.encode('latin-1').decode('utf-8')
    except UnicodeError:
        return data
    
def if_in_english(data):
    try:
        data.encode('ascii')
    except UnicodeEncodeError:
        return False
    except UnicodeError:
        return False
    else:
        return True

In [ ]:
from database import Database
database = Database(
    'localhost',
    '26feb',
    'root',
    'root',
    'utf8mb4'
)
connection = database.connect_with_pymysql()

if connection:
    with connection.cursor() as cursor:
        try:
            for i in robi_qs:
                if not if_in_english(i['qbody']):
                    body = utf8_decode(i['qbody'])
                else:
                    body = i['qbody']
                sql = "insert into robi_qs(question_id, body, tag_id) values('"+str(i['qid'])+"', '"+body+"', '"+str(i['qtag'])+"')"
                try:
                    cursor.execute(sql)
                    connection.commit()
                except Exception:
                    pass
        finally:
            connection.close()

In [ ]:
from database import Database
database = Database(
    'localhost',
    '26feb',
    'root',
    'root',
    'utf8mb4'
)
connection = database.connect_with_pymysql()

if connection:
    with connection.cursor() as cursor:
        try:
            sql = "select count(*) as count,body from answers where question_id in(select id from questions where source='robi') group by body order by count desc"
            cursor.execute(sql)
            data = cursor.fetchall()
            for i in data:
                cursor.execute("insert into robi_answers(count, answer) values('"+str(i['count'])+"','"+i['body']+"')")
                connection.commit()            
        finally:
            connection.close()

In [ ]:
from database import Database
database = Database(
    'localhost',
    '26feb',
    'root',
    'root',
    'utf8mb4'
)
connection = database.connect_with_pymysql()

if connection:
    with connection.cursor() as cursor:
        try:
            sql = "select id,answer from robi_answers"
            cursor.execute(sql)
            data = cursor.fetchall()
            for i in data:
                cursor.execute("select question_id from answers where body='"+i['answer']+"'")
                q_id = cursor.fetchall()
                for j in q_id:
                    cursor.execute("insert into robi_qa(robi_ans_id,question_id) values('"+str(i['id'])+"','"+str(j['question_id'])+"')")
                    connection.commit()
        finally:
            connection.close()

In [1]:
# language separation
# bangla, english and banglish
import string
english_text = open('/Users/shuvo/Downloads/maya_article_en.txt').read()
english_text.translate(None, string.punctuation)
english_text = english_text.split(' ')
en_word_set = set()
for word in english_text:
    word = word.lower()
    en_word_set.add(word)
        
def language_detection(text):
    # input text as a string
    # bangla = 0
    # english = 1
    # banglish = 2
    try:
        s_text = text.decode("utf-8").replace(" ","")
    except UnicodeEncodeError:
        s_text = text.replace(" ","")
    size = len(s_text)
    bn_count, en_count = 0,0
    for letter in s_text:
        try:
            letter.encode('ascii')
        except UnicodeEncodeError:
            bn_count += 1
        except UnicodeError:
            bn_count += 1
        else:
            en_count += 1
    try:
        bn_prob = float(bn_count)/size
    except ZeroDivisionError:
        return 0
    if bn_prob > 0.60:
        return 0
    else:
        words = text.split(' ')
        no_words = len(words)
        en_word_count = 0
        for word in words:
            if word in en_word_set:
                en_word_count += 1
        en_prob = float(en_word_count)/no_words
        if en_prob > 0.60:
            return 1
        else: 
            return 2

# print language_detection("নিয়মিত ব্যায়াম করা পর্যাপ্ত পরিমানে ঘুম ও বিশ্রাম নেয়া নিয়মিত পুষ্টিকর খাদ্য খাওয়া সেক্স করার আগে বেশি করে foreplay করা আপনার smoking বা alcohol এর অভ্যাস থাকলে তা পরিহার করুন আপনি relaxation technique চেষ্টা করে দেখতে পারেন যৌন মিলনের সময় মাইন্ড কে একটু distract করার চেষ্টা করে দেখতে পারেন anxiety বা depression এ ভুগলেও এমন টা হতে পারে কোন কিছু নিয়ে বেশি দুশ্চিন্তা করবেন না")
# from database import Database

# database = Database(
#     'localhost',
#     '26feb',
#     'root',
#     'root',
#     'utf8mb4'
# )
# connection = database.connect_with_pymysql()

# if connection:
#     with connection.cursor() as cursor:
#         sql = "select id,answer from robi_answers"
#         cursor.execute(sql)
#         qs = cursor.fetchall()
# for i in qs:
#     lang = language_detection(i['answer'])
#     with connection.cursor() as cursor:
#         insert_sql = "update robi_answers set language='"+str(lang)+"' where id="+str(i['id'])+""
#         cursor.execute(insert_sql)
#         connection.commit()
# connection.close()

In [6]:
from database import Database

database = Database(
    'localhost',
    '26feb',
    'root',
    'root',
    'utf8mb4'
)
connection = database.connect_with_pymysql()

if connection:
    with connection.cursor() as cursor:
        sql = "select id, body from ques_beauty_care"
        cursor.execute(sql)
        qs = cursor.fetchall()
try:
    for i in qs:
        lang = language_detection(i['body'])
        with connection.cursor() as cursor:
            insert_sql = "update ques_beauty_care set lang="+str(lang)+" where id="+str(i['id'])
            cursor.execute(insert_sql)
            connection.commit()
finally:
    connection.close()

188 0
86 0
311 0
202 0
291 0
188 0
86 0
311 0
202 0
291 0
0 43
132 0
171 0
63 0
313 0
46 0
90 0
70 0
112 0
111 0
206 0
183 0
68 0
189 0
248 0
0 377
195 0
125 0
330 0
20 0
274 0
230 0
140 0
9 268
76 0
0 187
142 0
549 0
3 172
166 0
140 0
86 0
87 0
119 0
89 0
156 0
109 0
32 0
312 0
55 0
223 0
0 174
128 0
171 0
254 0
222 0
40 0
184 0
32 239
34 0
63 0
95 0
154 0
300 0
630 0
225 0
96 0
35 0
94 0
121 0
125 0
240 0
97 0
129 0
2 35
88 0
76 0
314 0
68 0
205 0
257 0
0 187
142 0
225 0
31 0
47 0
144 0
115 0
91 0
56 0
75 0
160 0
168 0
56 0
96 0
78 0
209 0
180 0
143 0
111 0
162 0
0 179
219 0
95 0
92 0
91 0
161 0
92 0
92 0
254 0
291 0
47 0
116 0
156 0
97 0
373 0
69 0
109 0
187 0
94 0
72 0
132 0
60 0
0 34
87 0
254 0
274 0
59 0
58 0
187 0
216 0
33 0
212 0
0 467
127 0
138 0
201 0
201 0
36 0
120 0
94 0
77 0
166 0
76 0
70 0
78 0
229 0
229 0
37 0
69 0
477 0
290 0
40 0
273 0
0 21
237 0
155 0
67 0
37 0
215 0
60 0
2 41
105 0
77 0
491 0
236 0
64 0
185 0
100 0
496 0
67 35
704 0
117 0
89 0
185 0
222 0
86 0
214 0
